In [1]:
import tensorflow as tf

In [2]:
v1 = tf.Variable(tf.constant(1., shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2., shape=[1]), name="v2")
result = v1 + v2
init_op = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, "model/model.ckpt")

In [3]:
# method 1
saver = tf.train.import_meta_graph("./model/model.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess, "./model/model.ckpt")
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
[3.]


In [3]:
# method 2
v1 = tf.Variable(tf.constant(1., shape=[1]), name="other-v1")
v2 = tf.Variable(tf.constant(1., shape=[1]), name="other-v2")
result = v1 + v2
saver = tf.train.Saver({"v1": v1, "v2": v2})
with tf.Session() as sess:
    saver.restore(sess, "./model/model.ckpt")
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
[3.]


******

In [1]:
import tensorflow as tf

In [2]:
v = tf.Variable(0.0, tf.float32, name="v")
for variable in tf.global_variables():
    print(variable.name)

ema = tf.train.ExponentialMovingAverage(0.99)
ema_op = ema.apply(tf.global_variables())

for variable in tf.global_variables():
    print(variable.name)
    
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    sess.run(tf.assign(v, 10))
    sess.run(ema_op)
    saver.save(sess, "./model1/model.ckpt")
    print(sess.run([v, ema.average(v)]))

v:0
v:0
v/ExponentialMovingAverage:0
[10.0, 0.099999905]


In [2]:
# method 1
import tensorflow as tf

v = tf.Variable(0., tf.float32, name="v")
saver = tf.train.Saver({"v/ExponentialMovingAverage": v})
with tf.Session() as sess:
    saver.restore(sess, "./model1/model.ckpt")
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from ./model1/model.ckpt
0.099999905


******

In [1]:
# method 2
import tensorflow as tf

v = tf.Variable(0., tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)
saver = tf.train.Saver(ema.variables_to_restore())
with tf.Session() as sess:
    saver.restore(sess, "./model1/model.ckpt")
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from ./model1/model.ckpt
0.099999905


********

In [1]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

In [2]:
v1 = tf.Variable(tf.constant(1., shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(1., shape=[1]), name="v2")
result = v1 + v2
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ["add"])
    with tf.gfile.GFile("model2/combined_model.pb", "wb") as f:
        f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


In [1]:
import tensorflow as tf
from tensorflow.python.platform import gfile

In [2]:
with tf.Session() as sess:
    with gfile.FastGFile("./model2/combined_model.pb", "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    
    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print(sess.run(result))

[array([2.], dtype=float32)]
